In [ ]:
from pathlib import Path

import numpy as np
from astropy.table import Table
import pandas as pd
import matplotlib.pyplot as plt
import mpl_scatter_density
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PolynomialFeatures

import torch
from torch.utils.data import DataLoader, TensorDataset, random_split
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint

import torch.nn as nn
import torch.optim as optim
import os

from plot_utils import plot_kiel_scatter_density

### Read in the cleaned APOGEE data and make a Kiel Diagram for the whole sample.

In [ ]:
apogee_path = Path("./data/apogee_cleaned.parquet")
apogee_cat = pd.read_parquet(apogee_path)


In [ ]:
fig_kiel, ax_kiel = plot_kiel_scatter_density(
    apogee_cat['TEFF'],
    apogee_cat['LOGG'],
    apogee_cat['FE_H']
)

### Train-test split and fit a simple linear model

In [ ]:
# Create simple linear model to predict feh from teff and logg
# perform train test split and normalize the data


X = apogee_cat[['TEFF', 'LOGG']]
y = apogee_cat['FE_H']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.9, random_state=42)

# Normalize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

#Print the sizes of the training and testing sets
print(f"Training set size: {X_train_scaled.shape[0]}")
print(f"Testing set size: {X_test_scaled.shape[0]}")

In [ ]:
model = LinearRegression()
model.fit(X_train_scaled, y_train)
# Predict FE_H values
predicted_feh = model.predict(X_test_scaled)

# Plot the kiel diagram for the predicted values (test set only)
fig_kiel, ax_kiel = plot_kiel_scatter_density(
    X_test['TEFF'],
    X_test['LOGG'], 
    predicted_feh,
    title='Kiel Diagram - Linear model',
    colorbar_label='Predicted [Fe/H]',
)

#Print the slope and intercept of the linear model
print(f"Slope: {model.coef_}, Intercept: {model.intercept_}")

### Polynomial Regression (with Linear Algebra Optimizer)

In [ ]:

def polynomial_regression_lin_alg(X_train_scaled, X_test_scaled, y_train, test_teff, test_logg, degree):
    """
    Fits a polynomial regression model and plots the Kiel diagram with predicted values.
    
    Args:
        X_train_scaled: Scaled training features
        X_test_scaled: Scaled test features  
        y_train: Training target values
        test_teff: Test set effective temperatures
        test_logg: Test set surface gravities
        degree: Degree of polynomial features
        
    Returns:
        tuple: (poly_model, predicted_feh_poly, fig, ax, density_map)
    """
    # Create polynomial features
    poly = PolynomialFeatures(degree=degree, include_bias=True)
    # Transform the training and testing data
    X_train_poly = poly.fit_transform(X_train_scaled)
    X_test_poly = poly.transform(X_test_scaled)
    
    # Fit the polynomial regression model
    poly_model = LinearRegression(fit_intercept=False)
    poly_model.fit(X_train_poly, y_train)
    
    # Predict FE_H values using the polynomial model
    predicted_feh_poly = poly_model.predict(X_test_poly)

    return poly_model, predicted_feh_poly



In [ ]:
# Generate polynomial regression plots for different degrees
polynomial_degrees = [2, 3]
models = {}

for degree in polynomial_degrees:
    print(f"\nGenerating polynomial regression of degree {degree}...")
    model, predictions = polynomial_regression_lin_alg(
        X_train_scaled, X_test_scaled, y_train, X_test["TEFF"], X_test["LOGG"], degree
    )
    # Plot the kiel diagram for the predicted values (test set only)
    fig, ax = plot_kiel_scatter_density(
        X_test["TEFF"],
        X_test["LOGG"], 
        predictions,
        title=f'Polynomial regression of degree {degree} | Number of parameters: {len(model.coef_)}',
        colorbar_label='Predicted [Fe/H]',
    )

### Polynomial Regression with Gradient Descent Optimizer

In [ ]:
class LinearRegressionGD(pl.LightningModule):
    def __init__(self, input_dim, output_dim=1, **kwargs):
        super().__init__()
        self.save_hyperparameters() # Saves degree, input_dim, learning_rate

        self.linear_layer = nn.Linear(input_dim, output_dim, bias=False) # Output is 1 (e.g., FE_H)
    def forward(self, x_batch_tensor):
        return self.linear_layer(x_batch_tensor)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = nn.functional.mse_loss(y_hat, y)
        self.log('train_loss', loss, on_step=False, on_epoch=True, prog_bar=False)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = nn.functional.mse_loss(y_hat, y)
        self.log('val_loss', loss, on_step=False, on_epoch=True, prog_bar=True)
        return loss

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = nn.functional.mse_loss(y_hat, y)
        self.log('test_loss', loss, on_step=False, on_epoch=True)
        return loss

    def configure_optimizers(self):
        optimizer = optim.SGD(self.parameters(), lr=self.hparams.learning_rate)
        return optimizer

In [ ]:
def polynomial_regression_gradient_descent(X_train_scaled, X_test_scaled, y_train, test_teff, test_logg, degree, **kwargs):
       
    # Hyperparameters from kwargs with defaults
    learning_rate = kwargs.get('learning_rate', 1e-3)
    max_epochs = kwargs.get('max_epochs', 1000)
    batch_size = kwargs.get('batch_size', -1)  # -1 means use full batch
    patience = kwargs.get('patience', 10)
    val_split_ratio = kwargs.get('val_split_ratio', 0.2)
    num_workers = kwargs.get('num_workers', 0)
    accelerator = kwargs.get('accelerator', 'auto')
    devices = kwargs.get('devices', 'auto')
    checkpoint_dir = kwargs.get('checkpoint_dir', f'poly_regression_pt_checkpoints/deg_{degree}/')

    os.makedirs(checkpoint_dir, exist_ok=True)
    if batch_size <= 0:
        batch_size = len(X_train_scaled)
    # Create polynomial features
    poly = PolynomialFeatures(degree=degree, include_bias=True)
    # Transform the training and testing data
    X_train_poly = poly.fit_transform(X_train_scaled)
    X_test_poly = poly.transform(X_test_scaled)

    # Prepare data: Convert numpy arrays and pandas Series to PyTorch Tensors
    X_train_tensor = torch.tensor(X_train_poly, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train.values.reshape(-1, 1), dtype=torch.float32)
    X_test_tensor = torch.tensor(X_test_poly, dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test.values.reshape(-1, 1), dtype=torch.float32)

    # Create full training dataset
    full_train_dataset = TensorDataset(X_train_tensor, y_train_tensor)

    # Split full training dataset into train and validation sets
    num_train_samples = len(full_train_dataset)
    num_val_samples = int(val_split_ratio * num_train_samples)
    num_actual_train_samples = num_train_samples - num_val_samples

    train_dataset, val_dataset = random_split(
        full_train_dataset, [num_actual_train_samples, num_val_samples],
        generator=torch.Generator().manual_seed(42) # for reproducibility
    )

    persistent_workers_flag = num_workers > 0
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers, persistent_workers=persistent_workers_flag)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, num_workers=num_workers, persistent_workers=persistent_workers_flag)
    test_loader = DataLoader(TensorDataset(X_test_tensor, y_test_tensor), batch_size=batch_size, num_workers=num_workers, persistent_workers=persistent_workers_flag)
    # Callbacks
    early_stop_callback = EarlyStopping(monitor='val_loss', patience=patience, verbose=False, mode='min')
    checkpoint_callback = ModelCheckpoint(
        monitor='val_loss',
        dirpath=checkpoint_dir,
        filename=f'poly_deg{degree}-{{epoch:02d}}-{{val_loss:.4f}}',
        save_top_k=1,
        mode='min',
    )

    # Initialize model
    model = LinearRegressionGD(
        input_dim=X_train_poly.shape[1],  # Number of polynomial features
        output_dim=1,  # Predicting a single value (e.g., FE_H)
        learning_rate=learning_rate
    )

    # Initialize Trainer
    trainer = pl.Trainer(
        max_epochs=max_epochs,
        callbacks=[early_stop_callback, checkpoint_callback],
        accelerator=accelerator,
        devices=devices,
        logger=False, # Uses TensorBoardLogger by default should move to WandB.
        enable_progress_bar=True,
        deterministic=True # For reproducibility
    )
    pl.seed_everything(42, workers=True) # For reproducibility

    # Train the model
    print(f"\nTraining PyTorch Polynomial Regression (degree {degree})...")
    trainer.fit(model, train_dataloaders=train_loader, val_dataloaders=val_loader)

    # Load the best model from checkpoint
    if checkpoint_callback.best_model_path:
        print(f"Loading best model from: {checkpoint_callback.best_model_path}")
        best_model = LinearRegressionGD.load_from_checkpoint(checkpoint_callback.best_model_path)
        best_model.eval() # Set to evaluation mode
    else:
        print("No best model path found from checkpoint callback. Using the model instance after fit.")
        best_model = model # Fallback, though best_model_path should ideally exist
        best_model.eval()


    # Make predictions on the test set
    predicted_feh_test = torch.concat(trainer.predict(best_model, dataloaders=DataLoader(X_test_tensor, batch_size=batch_size)))
    predicted_feh_train = torch.concat(trainer.predict(best_model, dataloaders=DataLoader(X_train_tensor, batch_size=batch_size)))

    train_loss = nn.functional.mse_loss(predicted_feh_train, y_train_tensor).item()
    test_loss = nn.functional.mse_loss(predicted_feh_test, y_test_tensor).item()

    return best_model, np.squeeze(predicted_feh_test.numpy()), train_loss, test_loss

In [ ]:
model, predicted_feh,  train_loss, test_loss = polynomial_regression_gradient_descent(
        X_train_scaled, X_test_scaled, y_train, X_test["TEFF"], X_test["LOGG"], degree=1,
        learning_rate=1e-2,
        max_epochs=10
    )
# print the training and test loss
print(f"Training loss: {train_loss:.4f}, Test loss: {test_loss:.4f}")
total_model_params = sum(p.numel() for p in model.parameters())
print(f"Total model parameters: {total_model_params}")

In [ ]:
# Plot the Kiel diagram
fig, ax  = plot_kiel_scatter_density(
    X_test["TEFF"],
    X_test["LOGG"], 
    np.squeeze(predicted_feh.numpy()),
    title=f'Polynomial regression of degree {degree} | Number of parameters: {total_model_params}',
    colorbar_label='Predicted [Fe/H]',
)

# TO BE Moved to a separate notebook on DL

In [ ]:


# Define the MLP model using PyTorch Lightning
class MLP(pl.LightningModule):
    def __init__(self, network_shape, input_dim, learning_rate=1e-3, output_dim=1):
        super().__init__()
        # Save hyperparameters (network_shape, input_dim, learning_rate, output_dim)
        # This allows loading from checkpoint without re-passing these args
        self.save_hyperparameters()

        layers = []
        current_dim = self.hparams.input_dim
        for hidden_dim in self.hparams.network_shape:
            layers.append(nn.Linear(current_dim, hidden_dim))
            layers.append(nn.ReLU())
            current_dim = hidden_dim
        layers.append(nn.Linear(current_dim, self.hparams.output_dim))
        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = nn.functional.mse_loss(y_hat, y)
        self.log('train_loss', loss, on_step=False, on_epoch=True, prog_bar=False)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = nn.functional.mse_loss(y_hat, y)
        self.log('val_loss', loss, on_step=False, on_epoch=True, prog_bar=True)
        return loss

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = nn.functional.mse_loss(y_hat, y)
        self.log('test_loss', loss, on_step=False, on_epoch=True)
        return loss

    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=self.hparams.learning_rate)
        return optimizer

# Function to fit MLP, make predictions, and plot Kiel diagram
def fit_and_plot_mlp_regression(
    X_train_scaled_np, X_test_scaled_np, y_train_series, y_test_series,
    test_teff_series, test_logg_series, network_shape, **kwargs
):
    """
    Fits an MLP regression model using PyTorch Lightning, plots the Kiel diagram with predicted values.

    Args:
        X_train_scaled_np (np.ndarray): Scaled training features.
        X_test_scaled_np (np.ndarray): Scaled test features.
        y_train_series (pd.Series): Training target values.
        y_test_series (pd.Series): Test target values.
        test_teff_series (pd.Series): Test set effective temperatures.
        test_logg_series (pd.Series): Test set surface gravities.
        network_shape (list of int): List defining the number of neurons in each hidden layer.
        **kwargs: Additional hyperparameters for training.
            learning_rate (float): Optimizer learning rate. Default: 1e-3.
            batch_size (int): Batch size for DataLoaders. Default: 256.
            max_epochs (int): Maximum number of training epochs. Default: 100.
            patience (int): Patience for EarlyStopping. Default: 10.
            val_split_ratio (float): Fraction of training data to use for validation. Default: 0.2.
            num_workers (int): Number of workers for DataLoader. Default: 0.
            accelerator (str): PyTorch Lightning accelerator ('cpu', 'gpu', 'auto'). Default: 'auto'.
            devices (any): PyTorch Lightning devices. Default: 'auto'.
            checkpoint_dir (str): Directory to save model checkpoints. Default: 'mlp_checkpoints/'.

    Returns:
        tuple: (best_model, predicted_feh_mlp, fig, ax, density_map)
            best_model (MLP): The trained MLP model (best checkpoint).
            predicted_feh_mlp (np.ndarray): Predicted [Fe/H] values on the test set.
            fig (matplotlib.figure.Figure): The figure object for the Kiel diagram.
            ax (matplotlib.axes.Axes): The axes object for the Kiel diagram.
            density_map (matplotlib.collections.PathCollection): The scatter density plot object.
    """
    # Hyperparameters from kwargs with defaults
    learning_rate = kwargs.get('learning_rate', 1e-3)
    batch_size = kwargs.get('batch_size', 256)
    max_epochs = kwargs.get('max_epochs', 100)
    patience = kwargs.get('patience', 10)
    val_split_ratio = kwargs.get('val_split_ratio', 0.2)
    num_workers = kwargs.get('num_workers', 0) # Default to 0 for broader compatibility
    accelerator = kwargs.get('accelerator', 'auto')
    devices = kwargs.get('devices', 'auto')
    checkpoint_dir = kwargs.get('checkpoint_dir', 'mlp_checkpoints/')
    
    # Ensure checkpoint directory exists
    os.makedirs(checkpoint_dir, exist_ok=True)

    # Prepare data: Convert numpy arrays and pandas Series to PyTorch Tensors
    X_train_tensor = torch.tensor(X_train_scaled_np, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train_series.values.reshape(-1, 1), dtype=torch.float32)
    X_test_tensor = torch.tensor(X_test_scaled_np, dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test_series.values.reshape(-1, 1), dtype=torch.float32)

    # Create full training dataset
    full_train_dataset = TensorDataset(X_train_tensor, y_train_tensor)

    # Split full training dataset into actual train and validation sets
    num_train_samples = len(full_train_dataset)
    num_val_samples = int(val_split_ratio * num_train_samples)
    num_actual_train_samples = num_train_samples - num_val_samples

    train_dataset, val_dataset = random_split(
        full_train_dataset, [num_actual_train_samples, num_val_samples],
        generator=torch.Generator().manual_seed(42) # for reproducibility
    )
    
    persistent_workers_flag = num_workers > 0
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers, persistent_workers=persistent_workers_flag)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, num_workers=num_workers, persistent_workers=persistent_workers_flag)
    
    test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, num_workers=num_workers, persistent_workers=persistent_workers_flag)

    # Callbacks
    early_stop_callback = EarlyStopping(monitor='val_loss', patience=patience, verbose=False, mode='min')
    # Sanitize network_shape for filename
    shape_str = "_".join(map(str, network_shape))
    checkpoint_callback = ModelCheckpoint(
        monitor='val_loss',
        dirpath=checkpoint_dir,
        filename=f'mlp-shape_{shape_str}-{{epoch:02d}}-{{val_loss:.4f}}',
        save_top_k=1,
        mode='min',
    )

    # Initialize model
    model = MLP(
        network_shape=network_shape,
        input_dim=X_train_scaled_np.shape[1],
        learning_rate=learning_rate
    )


    # Initialize Trainer
    trainer = pl.Trainer(
        max_epochs=max_epochs,
        callbacks=[early_stop_callback, checkpoint_callback],
        accelerator=accelerator,
        devices=devices,
        logger=True, # Uses TensorBoardLogger by default, logs to lightning_logs/
        enable_progress_bar=True,
        deterministic=True # For reproducibility, might impact performance
    )
    pl.seed_everything(42, workers=True) # For reproducibility

    # Train the model
    print(f"\nTraining MLP with network shape: {network_shape}...")
    trainer.fit(model, train_dataloaders=train_loader, val_dataloaders=val_loader)

    # Load the best model from checkpoint
    print(f"Loading best model from: {checkpoint_callback.best_model_path}")
    best_model = MLP.load_from_checkpoint(checkpoint_callback.best_model_path)
    best_model.eval() # Set to evaluation mode

    # Evaluate on the test set (optional, but good practice)
    test_results = trainer.test(best_model, dataloaders=test_loader, verbose=False)
    print(f"Test results for MLP shape {network_shape}: {test_results}")

    # Make predictions on the test set
    with torch.no_grad():
        predicted_feh_mlp_tensor = best_model(X_test_tensor)
    predicted_feh_mlp = predicted_feh_mlp_tensor.cpu().numpy().flatten()

    # Plot the Kiel diagram using the existing plot_kiel_scatter_density function
    fig, ax, density_map = plot_kiel_scatter_density(
        test_teff_series,
        test_logg_series,
        predicted_feh_mlp,
        title=f'Kiel Diagram - MLP Regression (Shape: {network_shape})',
        colorbar_label='Predicted [Fe/H] (MLP)',
    )
    plt.show()

    return best_model, predicted_feh_mlp, fig, ax, density_map


# Example usage: Define network shapes and run the MLP regression function
# These variables are assumed to be defined in previous cells:
# X_train_scaled, X_test_scaled, y_train, y_test, test_teff, test_logg
# plot_kiel_scatter_density (function)

mlp_network_shapes = [
    [64],             # Single hidden layer with 64 neurons
    [64, 32],         # Two hidden layers: 64 neurons, then 32 neurons
    [128, 64, 32]     # Three hidden layers
]
mlp_results = {}

# Set common hyperparameters for all MLP runs, can be overridden in kwargs
mlp_hyperparams = {
    'learning_rate': 0.001,
    'batch_size': 512,
    'max_epochs': 50,  # Adjust as needed, can be short for demonstration
    'patience': 5,     # Adjust as needed
    'num_workers': 0,  # Set to 0 for simplicity, os.cpu_count() for performance if env supports
    'accelerator': 'gpu', 'devices': 1, # Uncomment if GPU is available
}

for shape in mlp_network_shapes:
    print("-" * 50)
    shape_key = f'shape_{"_".join(map(str, shape))}'
    
    model_mlp, predictions_mlp, fig_mlp, ax_mlp, dm_mlp = fit_and_plot_mlp_regression(
        X_train_scaled, X_test_scaled, y_train, y_test,
        test_teff, test_logg,
        network_shape=shape,
        **mlp_hyperparams 
    )
    mlp_results[shape_key] = {
        'model': model_mlp,
        'predictions': predictions_mlp,
        'fig': fig_mlp,
        'ax': ax_mlp,
        'density_map': dm_mlp
    }

print("-" * 50)
print("MLP regression experiments complete.")
print(f"Results stored in mlp_results dictionary with keys: {list(mlp_results.keys())}")

In [ ]:

# Example usage for PyTorch Polynomial Regression
# Assumes X_train_scaled, X_test_scaled, y_train, y_test, test_teff, test_logg are defined

poly_pt_degrees = [2, 3, 5]  # Degrees to test, 5 instead of 10 for potentially faster run
poly_pt_results = {}

# Common hyperparameters for PyTorch Polynomial Regression runs
poly_pt_hyperparams = {
    'learning_rate': 0.01,     # Polynomial regression might benefit from a slightly higher LR
    'batch_size': 512,
    'max_epochs': 100,         # Can be adjusted; set lower for quicker tests
    'patience': 10,            # For early stopping
    'num_workers': 0,          # Set to 0 for simplicity, can be os.cpu_count() for performance
    'accelerator': 'gpu' if torch.cuda.is_available() else 'cpu',
    'devices': 1 if torch.cuda.is_available() else 'auto',
    # checkpoint_dir is handled within fit_and_plot_poly_regression_pytorch
}

print("Starting PyTorch Polynomial Regression experiments...")
for degree_val in poly_pt_degrees:
    print("-" * 50)
    degree_key = f'degree_{degree_val}'
    
    # Ensure all required variables are available in the notebook's global scope
    # These should have been defined in cells above where X, y were split and scaled.
    model_poly_pt, predictions_poly_pt, fig_poly_pt, ax_poly_pt, dm_poly_pt = fit_and_plot_poly_regression_pytorch(
        X_train_scaled, X_test_scaled, y_train, y_test,
        test_teff, test_logg, # These are pd.Series for Teff and Logg from the X_test dataframe
        degree=degree_val,
        **poly_pt_hyperparams 
    )
    poly_pt_results[degree_key] = {
        'model': model_poly_pt,
        'predictions': predictions_poly_pt,
        'fig': fig_poly_pt,
        'ax': ax_poly_pt,
        'density_map': dm_poly_pt
    }

print("-" * 50)
print("PyTorch Polynomial Regression experiments complete.")
print(f"Results stored in poly_pt_results dictionary with keys: {list(poly_pt_results.keys())}")
